In [1]:
import time

import kscope

### Conecting to the Service
First we connect to the Kaleidoscope service through which we'll interact with the LLMs and see which models are avaiable to us

In [2]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)

Show all model instances that are currently active

In [3]:
client.model_instances

[{'id': '69765f91-ecc1-4d7f-a3a4-b18ed38370c4',
  'name': 'falcon-40b',
  'state': 'ACTIVE'}]

To start, we obtain a handle to a model. In this example, let's use the Falcon-40B model.

In [4]:
model = client.load_model("falcon-40b")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

In [5]:
moderate_generation_config = {"max_tokens": 100, "top_k": 1, "top_p": 1.0, "temperature": 0.8}

Let's ask the model some questions

In [7]:
generation = model.generate("What is the capital of Canada?", moderate_generation_config)
# Extract the text from the returned generation
print(generation.generation["sequences"][0])

Ottawa


# Few-Shot Chain of Thought Prompting

We'll start by prompting Falcon-40B to solve some word problems using the Few-Shot CoT method proposed in ["Chain-of-Thought Prompting Elicits Reasoning
in Large Language Models"](https://arxiv.org/pdf/2201.11903.pdf)

First, let's see what happens if we try to solve some word problems with a zero-shot prompt.

In [8]:
zero_shot_prompt = (
    "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"
)

print(zero_shot_prompt)

The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples are left?


In [9]:
generation_example = model.generate(zero_shot_prompt, generation_config=moderate_generation_config)
print(generation_example.generation["sequences"][0])

Get your free account and access expert answers to this and thousands of other questions


The correct answer to this word problem is 9.

Now let's try performing a few-shot prompt to see if that helps the model provide the correct answer in a format that we can extract.

In [10]:
few_shot_prompt = (
    "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis "
    "balls does he have now?\nA: The answer is 11.\n\nQ: The cafeteria had 23 apples. If they used 20 to make lunch "
    "and bought 6 more, how many apples do they have?\nA:"
)
print(few_shot_prompt)

Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
A:


In [11]:
generation_example = model.generate(few_shot_prompt, generation_config=moderate_generation_config)
print(generation_example.generation["sequences"][0])

The answer is 29.


Now, let's try prompting the model with a few-shot CoT prompt, where we provide an example of the kind of reasoning required to answer the question.

In [12]:
few_shot_cot_prompt = (
    "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis "
    "balls does he have now?\nA: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. "
    "5 + 6 = 11. The answer is 11.\n\nQ: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 "
    "more, how many apples do they have?\nA:"
)
print(few_shot_cot_prompt)

Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
A:


In [13]:
generation_example = model.generate(few_shot_cot_prompt, generation_config=moderate_generation_config)
print(generation_example.generation["sequences"][0])

23 - 20 = 3. 3 + 6 = 9. The answer is 9.


Let's try compare few-shot prompting with few-shot CoT for slightly different kind of problem. This example is drawn from the AQuA: Algebraic Word Problems task.

In [14]:
few_shot_prompt = (
    "Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of "
    "the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64\nA: The answer is (a).\n\nQ: The capacity of "
    "a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) "
    "120000 litres (e) None of these\nA:"
)
print(few_shot_prompt)

Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64
A: The answer is (a).

Q: The capacity of a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) 120000 litres (e) None of these
A:


In [15]:
generation_example = model.generate(few_shot_prompt, generation_config=moderate_generation_config)
print(generation_example.generation["sequences"][0])

 The answer is (b).

Q: The average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64
A: The answer is (a).

Q: The capacity of a tank of dimensions (8 m × 6 m ×


The correct choice for this problem is (d)

Let's see if we can extract the correct answer with few-shot CoT

In [17]:
few_shot_cot_prompt = (
    "Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers "
    "is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64\nA: If 10 is added to each number, then the mean of the "
    "numbers also increases by 10. So the new mean would be 50. The answer is (a).\n\nQ: The capacity of "
    "a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) "
    "120000 litres (e) None of these \nA:"
)
print(few_shot_cot_prompt)

Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64
A: If 10 is added to each number, then the mean of the numbers also increases by 10. So the new mean would be 50. The answer is (a).

Q: The capacity of a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) 120000 litres (e) None of these 
A:


In [18]:
generation_example = model.generate(few_shot_cot_prompt, generation_config=moderate_generation_config)
print(generation_example.generation["sequences"][0])

 The capacity of a tank of dimensions (8 m × 6 m × 2.5 m) is 120000 litres. The answer is (d).

Q: The average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64 



# Zero-Shot Chain of Thought Prompting

It can be tedious and tricky to form useful and effective reasoning examples. Some research has shown that the choice of reasoning examples in CoT prompting can have a large impact on how well the model accomplishes the downstream task. So let's try a zero-shot CoT approach devised in ["Large Language Models are Zero-Shot Reasoners"](https://arxiv.org/pdf/2205.11916.pdf)

In [19]:
few_shot_prompt = (
    "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis "
    "balls does he have now?\nA: The answer is 11.\n\nQ: There are 64 students trying out for the school's trivia "
    "teams. If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many students would "
    "be in each group?\nA:"
)
print(few_shot_prompt)

Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.

Q: There are 64 students trying out for the school's trivia teams. If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many students would be in each group?
A:


In [20]:
generation_example = model.generate(few_shot_prompt, generation_config=moderate_generation_config)
print(generation_example.generation["sequences"][0])

 The answer is 16.

Q: There are 3 people in a room. One of them is a murderer. The other two are detectives. The murderer is lying. The detectives are telling the truth. The murderer says, "I am lying." The first detective says, "I am lying." The second detective says, "I am telling the truth." Who is the murderer?
A: The answer is the first detective.

Q: A man is


The correct answer to this problem is 7.

Perhaps we can extract the correct answer with zero-Shot CoT is split into two stages:
1) Reasoning Generation
2) Answer Extraction

In [21]:
reasoning_generation_prompt = (
    "Q: There are 64 students trying out for the school's trivia teams. If 36 of them didn't get picked for the team "
    "and the rest were put into 4 groups, how many students would be in each group?\nA: Let’s think step by step."
)
print(reasoning_generation_prompt)

Q: There are 64 students trying out for the school's trivia teams. If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many students would be in each group?
A: Let’s think step by step.


In [22]:
reasoning_generation = model.generate(
    reasoning_generation_prompt, generation_config=moderate_generation_config
).generation["sequences"][0]
print(reasoning_generation)


First, we need to find out how many students are in each group.
We know that there are 64 students trying out for the school’s trivia teams.
We also know that 36 of them didn’t get picked for the team.
So, we can subtract 36 from 64 to find out how many students were picked for the team.
64 – 36 = 28
Now, we need to divide 28 by 4 to find


In [23]:
answer_extraction_prompt = f"{reasoning_generation_prompt}{reasoning_generation}\nTherefore, the answer is"
print(answer_extraction_prompt)

Q: There are 64 students trying out for the school's trivia teams. If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many students would be in each group?
A: Let’s think step by step.
First, we need to find out how many students are in each group.
We know that there are 64 students trying out for the school’s trivia teams.
We also know that 36 of them didn’t get picked for the team.
So, we can subtract 36 from 64 to find out how many students were picked for the team.
64 – 36 = 28
Now, we need to divide 28 by 4 to find
Therefore, the answer is


In [24]:
answer_generation = model.generate(answer_extraction_prompt, generation_config=moderate_generation_config).generation[
    "sequences"
][0]
print(answer_generation)

 7.
Q: A school has 100 students. If 60 of them are girls and the rest are boys, how many boys are there?
A: Let’s think step by step.
First, we need to find out how many boys are in the school.
We know that there are 100 students in the school.
We also know that 60 of them are girls.
So, we can subtract 60 from 100 to find out how many
